In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import json
import requests
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import math
import statistics
import sys

In [2]:
draft_df = pd.read_excel("C:\\Users\\mcyee\\Desktop\\yahoo_keeper_2021\\2021_Fantasy_Football_Draft.xlsx")
draft_df.head()

,pick,player_name,player_position,player_team,salary,manager,keeper_code
0,1,Aaron Jones,RB,GB,27,Matt,BCDE
1,2,Jonathan Taylor,RB,IND,64,Matt,E
2,3,Odell Beckham Jr.,WR,CLE,5,Matt,E
3,4,Jerry Jeudy,WR,DEN,5,Matt,E
4,5,Davante Adams,WR,GB,60,Dai,ABCDE


In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')

In [4]:
transaction_list = []
driver = webdriver.Chrome('C:\\Users\\mcyee\\Desktop\\yahoo_keeper_2021\\chromedriver.exe')

for n in np.arange(0, 20):
    try:
        p = str(n*25)
        driver.get("https://football.fantasysports.yahoo.com/f1/597209/transactions?transactionsfilter=all&count=" + p)
        time.sleep(2)
        transaction_html = driver.page_source
        transaction_soup = BeautifulSoup(transaction_html, "html.parser")
        transaction_table = transaction_soup.findAll("table", {"class": "Table Table-mid Tst-transaction-table"})[0].findAll("tbody")[0]
        for tr in transaction_table.findAll('tr'):
            td = tr.findAll('td')
            if str(td[0]).find("Added Player") != -1 and str(td[0]).find("Dropped Player") != -1:
                player_added = td[1].find_all("a", {'target': 'sports'})[0].text
                player_added_team = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[0]
                player_added_pos = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[1]
                player_dropped = td[1].find_all("a", {'target': 'sports'})[1].text
                player_dropped_team = td[1].find_all('span', {'class': 'F-position'})[1].text.split(' - ')[0]
                player_dropped_pos = td[1].find_all('span', {'class': 'F-position'})[1].text.split(' - ')[1]
                manager = td[2].find_all("a", {'class': 'Tst-team-name'})[0].text
                dt_transaction = td[2].find_all("span", {'class': 'F-timestamp'})[0].text
                transaction_details = td[1].find_all('h6', {'class': 'F-shade'})[0].text
                if transaction_details.find("$") != -1:
                    transaction_cost = (transaction_details.replace('$', '').replace(' ', '').replace('Waiver', ''))
                else:
                    transaction_cost = 0    
                transaction_dict = {
                    'transaction_type': 'drop-add',
                    'player_added': player_added,
                    'player_added_team': player_added_team,
                    'player_added_pos': player_added_pos,
                    'player_dropped': player_dropped,
                    'player_dropped_team': player_dropped_team,
                    'player_dropped_pos': player_dropped_pos,
                    'ff_team': manager,
                    'transaction_time': dt_transaction,
                    'transaction_cost': transaction_cost
                }
                transaction_list.append(transaction_dict)
            elif str(td[0]).find("Dropped Player") != -1:
                player_dropped = td[1].find_all("a", {'target': 'sports'})[0].text
                player_dropped_team = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[0]
                player_dropped_pos = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[1]
                manager = td[2].find_all("a", {'class': 'Tst-team-name'})[0].text
                dt_transaction = td[2].find_all("span", {'class': 'F-timestamp'})[0].text
                transaction_dict = {
                    'transaction_type': 'drop',
                    'player_added': 'Bench',
                    'player_added_team': 'N/A',
                    'player_added_pos': 'N/A',
                    'player_dropped': player_dropped,
                    'player_dropped_team': player_dropped_team,
                    'player_dropped_pos': player_dropped_pos,
                    'ff_team': manager,
                    'transaction_time': dt_transaction,
                    'transaction_cost': 0
                }
                transaction_list.append(transaction_dict)
            elif str(td[0]).find("Added Player") != -1:
                player_added = td[1].find_all("a", {'target': 'sports'})[0].text
                player_added_team = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[0]
                player_added_pos = td[1].find_all('span', {'class': 'F-position'})[0].text.split(' - ')[1]
                manager = td[2].find_all("a", {'class': 'Tst-team-name'})[0].text
                dt_transaction = td[2].find_all("span", {'class': 'F-timestamp'})[0].text
                transaction_details = td[1].find_all('h6', {'class': 'F-shade'})[0].text
                if transaction_details.find("$") != -1:
                    transaction_cost = (transaction_details.replace('$', '').replace(' ', '').replace('Waiver', ''))
                else:
                    transaction_cost = 0 
                transaction_dict = {
                    'transaction_type': 'add',
                    'player_added': player_added,
                    'player_added_team': player_added_team,
                    'player_added_pos': player_added_pos,
                    'player_dropped': 'Bench',
                    'player_dropped_team': "N/A",
                    'player_dropped_pos': "N/A",
                    'ff_team': manager,
                    'transaction_time': dt_transaction,
                    'transaction_cost': transaction_cost
                }
                transaction_list.append(transaction_dict)

            elif str(td[1]).find("Traded to") != -1:
                manager = td[2].find_all("span", {'class': 'Grid-u'})[0].find_all("a")[0].text
                dt_transaction = td[2].find_all("span", {'class': 'F-timestamp'})[0].text
                adds = td[0].find_all('a', {'target': 'sports'})
                for a in range(len(adds)):
                    player_added = td[0].find_all("a", {'target': 'sports'})[a].text
                    player_added_team = td[0].find_all('span', {'class': 'F-position'})[a].text.split(' - ')[0]
                    player_added_pos = td[0].find_all('span', {'class': 'F-position'})[a].text.split(' - ')[1]
                    transaction_dict = {
                        'transaction_type': 'trade',
                        'player_added': player_added,
                        'player_added_team': player_added_team,
                        'player_added_pos': player_added_pos,
                        'player_dropped': 'N/A',
                        'player_dropped_team': "N/A",
                        'player_dropped_pos': "N/A",
                        'ff_team': manager,
                        'transaction_time': dt_transaction,
                        'transaction_cost': 0
                    }
                    transaction_list.append(transaction_dict)
            elif str(td[2]).find("Traded to") != -1:
                manager = td[3].find_all("span", {'class': 'Grid-u'})[0].find_all("a")[0].text
                dt_transaction = td[3].find_all("span", {'class': 'F-timestamp'})[0].text
                adds = td[1].find_all('a', {'target': 'sports'})
                for a in range(len(adds)):
                    player_added = td[1].find_all("a", {'target': 'sports'})[a].text
                    player_added_team = td[1].find_all('span', {'class': 'F-position'})[a].text.split(' - ')[0]
                    player_added_pos = td[1].find_all('span', {'class': 'F-position'})[a].text.split(' - ')[1]
                    transaction_dict = {
                        'transaction_type': 'trade',
                        'player_added': player_added,
                        'player_added_team': player_added_team,
                        'player_added_pos': player_added_pos,
                        'player_dropped': 'N/A',
                        'player_dropped_team': "N/A",
                        'player_dropped_pos': "N/A",
                        'ff_team': manager,
                        'transaction_time': dt_transaction,
                        'transaction_cost': 0
                    }
                    transaction_list.append(transaction_dict)
    except:
        pass

driver.quit()
transaction_df = pd.DataFrame(transaction_list)
transaction_df.head()

,ff_team,player_added,player_added_pos,player_added_team,player_dropped,player_dropped_pos,player_dropped_team,transaction_cost,transaction_time,transaction_type
0,folklore,Los Angeles,DEF,LAC,Philadelphia,DEF,Phi,0,"Nov 10, 11:02 am",drop-add
1,Odellta Variant,Le'Veon Bell,RB,Bal,Dan Arnold,TE,Jax,0,"Nov 10, 8:52 am",drop-add
2,Odellta Variant,Travis Etienne,RB,Jax,Boston Scott,RB,Phi,0,"Nov 10, 8:51 am",drop-add
3,Zeke and Ye Shall Find,Rhamondre Stevenson,RB,NE,Carlos Hyde,RB,Jax,0,"Nov 10, 6:43 am",drop-add
4,folklore,Philadelphia,DEF,Phi,San Francisco,DEF,SF,0,"Nov 10, 5:00 am",drop-add


In [5]:
manager_list = []
for m in list(set(transaction_df['ff_team'])):
    manager = input("Which player is the manager for " + m + "?")
    manager_dict = {
        'ff_team': m,
        'manager': manager
    }
    manager_list.append(manager_dict)
manager_df = pd.DataFrame(manager_list)
manager_df
    

Which player is the manager for Raider Bae?Jake
Which player is the manager for Tiz the Law?Dai
Which player is the manager for folklore?Ron
Which player is the manager for Pop Drop and Lockett?Evan
Which player is the manager for Me So Fourney?Joel
Which player is the manager for G?Jiwei
Which player is the manager for Odellta Variant?Matt
Which player is the manager for Nags?Ryan
Which player is the manager for Mediocre Team?Rajiv
Which player is the manager for Boswell that Ends Well?Sean
Which player is the manager for Chi ShingT's Team?Chi Shing
Which player is the manager for Zeke and Ye Shall Find?Doug


,ff_team,manager
0,Raider Bae,Jake
1,Tiz the Law,Dai
2,folklore,Ron
3,Pop Drop and Lockett,Evan
4,Me So Fourney,Joel
5,G,Jiwei
6,Odellta Variant,Matt
7,Nags,Ryan
8,Mediocre Team,Rajiv
9,Boswell that Ends Well,Sean


In [6]:
transaction_df = pd.DataFrame(transaction_list)
transaction_df['transaction_time'] = "2020 " + transaction_df['transaction_time']
transaction_df['transaction_time'] = pd.to_datetime(transaction_df['transaction_time'], format="%Y %b %d, %I:%M %p")
transaction_df = transaction_df.sort_values(by='transaction_time').reset_index(drop=True)
transaction_df = transaction_df.merge(manager_df, on='ff_team', how='left')
transaction_df.to_csv("C:\\Users\\mcyee\\Desktop\\yahoo_keeper_2021\\2021_Fantasy_Football_Transactions.csv")
transaction_df.head()

,ff_team,player_added,player_added_pos,player_added_team,player_dropped,player_dropped_pos,player_dropped_team,transaction_cost,transaction_time,transaction_type,manager
0,Pop Drop and Lockett,Ty'Son Williams,RB,Bal,Los Angeles,DEF,LAC,7,2020-08-30 00:09:00,drop-add,Evan
1,Me So Fourney,Jermar Jefferson,RB,Det,Kirk Cousins,QB,Min,5,2020-08-30 00:09:00,drop-add,Joel
2,Odellta Variant,Nelson Agholor,WR,NE,Todd Gurley II,RB,Atl,0,2020-08-30 00:09:00,drop-add,Matt
3,Boswell that Ends Well,Sterling Shepard,WR,NYG,Russell Gage,WR,Atl,0,2020-08-30 00:10:00,drop-add,Sean
4,Boswell that Ends Well,Tony Jones Jr.,RB,NO,Latavius Murray,RB,Bal,0,2020-08-30 00:13:00,drop-add,Sean


In [7]:
transaction_df.columns

Index(['ff_team', 'player_added', 'player_added_pos', 'player_added_team',
       'player_dropped', 'player_dropped_pos', 'player_dropped_team',
       'transaction_cost', 'transaction_time', 'transaction_type', 'manager'],
      dtype='object')

In [8]:
draft_df = pd.read_excel("C:\\Users\\mcyee\\Desktop\\yahoo_keeper_2021\\2021_Fantasy_Football_Draft.xlsx")
draft_df.head()

for i, r in transaction_df.iterrows():
#     print(i, r['transaction_type'], r['player_added'], r['player_added_team'], r['player_dropped'], r['player_dropped_team'])
    if r['transaction_type'] == 'drop-add':
        draft_row_n = draft_df[(draft_df['player_name'] == r['player_dropped']) & (draft_df['player_team'] == r['player_dropped_team'])]
        draft_df.at[draft_row_n.index[0], 'player_name'] = r['player_added']
        draft_df.at[draft_row_n.index[0], 'player_team'] = r['player_added_team']
        draft_df.at[draft_row_n.index[0], 'player_position'] = r['player_added_pos']
        draft_df.at[draft_row_n.index[0], 'salary'] = r['transaction_cost']
        draft_df = draft_df.reset_index(drop=True)
    elif r['transaction_type'] == 'drop':
        draft_row_n = draft_df[(draft_df['player_name'] == r['player_dropped']) & (draft_df['player_team'] == r['player_dropped_team'])]
        draft_df = draft_df.drop([draft_row_n.index[0]]).reset_index(drop=True)
    elif r['transaction_type'] == 'add':
        draft_df = draft_df.append({'pick': 0, 
                         'player_name': r['player_added'], 
                         'player_position': r['player_added_pos'], 
                         'player_team': r['player_added_team'],
                         'salary': r['transaction_cost'],
                         'manager': r['manager'],
                         'keeper_code': ''
                        }, ignore_index=True)
        draft_df = draft_df.reset_index(drop=True)
    elif r['transaction_type'] == 'trade':
        draft_row_n = draft_df[(draft_df['player_name'] == r['player_added']) & (draft_df['player_team'] == r['player_added_team'])]
        draft_df.at[draft_row_n.index[0], 'manager'] = r['manager']
        draft_df.reset_index(drop=True)
draft_df = draft_df.sort_values(by=['manager', 'salary']).reset_index(drop=True)

In [10]:
draft_df.to_csv('C:\\Users\\mcyee\\Desktop\\yahoo_keeper_2021\\yahoo_keeper_2021.csv')

In [23]:
draft_df[draft_df['manager'] == 'Chi Shing']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
0,96,C.J. Uzomah,TE,Cin,0,Chi Shing,NaN
1,132,Emmanuel Sanders,WR,Buf,0,Chi Shing,NaN
2,136,Khalil Herbert,RB,Chi,0,Chi Shing,NaN
3,157,Van Jefferson Jr.,WR,LAR,0,Chi Shing,NaN
4,145,Matt Gay,K,LAR,1,Chi Shing,NaN
5,86,Los Angeles,DEF,LAR,4,Chi Shing,NaN
6,129,Christian Kirk,WR,Ari,8,Chi Shing,NaN
7,126,DeVonta Smith,WR,Phi,12,Chi Shing,NaN
8,71,Melvin Gordon III,RB,Den,21,Chi Shing,NaN
9,10,CeeDee Lamb,WR,DAL,23,Chi Shing,E


In [24]:
draft_df[draft_df['manager'] == 'Dai']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
16,8,Jamal Agnew,WR,Jax,0,Dai,E
17,119,Derrick Gore,RB,KC,0,Dai,NaN
18,127,Greg Joseph,K,Min,0,Dai,NaN
19,138,Kadarius Toney,WR,NYG,0,Dai,NaN
20,139,Zach Ertz,TE,Ari,0,Dai,NaN
21,169,Rashaad Penny,RB,Sea,0,Dai,NaN
22,182,Zack Moss,RB,Buf,0,Dai,NaN
23,162,Michael Carter,RB,NYJ,3,Dai,NaN
24,122,Sony Michel,RB,LAR,9,Dai,NaN
25,40,Josh Allen,QB,Buf,23,Dai,E


In [25]:
draft_df[draft_df['manager'] == 'Doug']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
32,148,Buffalo,DEF,Buf,0,Doug,NaN
33,159,Michael Badgley,K,Ind,0,Doug,NaN
34,185,Chuba Hubbard,RB,Car,0,Doug,NaN
35,0,Rhamondre Stevenson,RB,NE,0,Doug,
36,0,Rondale Moore,WR,Ari,0,Doug,
37,168,Jakobi Meyers,WR,NE,1,Doug,NaN
38,27,Justin Herbert,QB,SD,5,Doug,E
39,29,James Robinson,RB,JAX,5,Doug,E
40,97,A.J. Green,WR,Ari,5,Doug,NaN
41,147,Mike Gesicki,TE,Mia,6,Doug,NaN


In [26]:
draft_df[draft_df['manager'] == 'Evan']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
48,73,Darrel Williams,RB,KC,0,Evan,NaN
49,121,Jimmy Garoppolo,QB,SF,0,Evan,NaN
50,133,Jared Cook,TE,LAC,0,Evan,NaN
51,152,Matt Prater,K,Ari,2,Evan,NaN
52,153,Darnell Mooney,WR,Chi,4,Evan,NaN
53,116,Michael Gallup,WR,Dal,11,Evan,NaN
54,128,Jarvis Landry,WR,Cle,15,Evan,NaN
55,85,Tyler Boyd,WR,Cin,18,Evan,NaN
56,77,Damien Harris,RB,NE,23,Evan,NaN
57,41,Kyler Murray,QB,ARI,24,Evan,DE


In [27]:
draft_df[draft_df['manager'] == 'Jake']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
64,25,Michael Thomas,WR,NO,0,Jake,E
65,83,DeSean Jackson,WR,LV,0,Jake,NaN
66,101,Jamison Crowder,WR,NYJ,0,Jake,NaN
67,146,Donovan Peoples-Jones,WR,Cle,0,Jake,NaN
68,106,Justin Tucker,K,Bal,2,Jake,NaN
69,112,Cole Beasley,WR,Buf,2,Jake,NaN
70,74,James Conner,RB,Ari,16,Jake,NaN
71,24,Cooper Kupp,WR,LAR,26,Jake,CDE
72,80,Mike Davis,RB,Atl,33,Jake,NaN
73,76,Josh Jacobs,RB,LV,38,Jake,NaN


In [28]:
draft_df[draft_df['manager'] == 'Jiwei']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
80,28,Alex Collins,RB,Sea,0,Jiwei,E
81,100,Chase McLaughlin,K,Cle,0,Jiwei,NaN
82,131,Kendrick Bourne,WR,NE,0,Jiwei,NaN
83,149,Brandon Bolden,RB,NE,0,Jiwei,NaN
84,160,D'Ernest Johnson,RB,Cle,0,Jiwei,NaN
85,137,Cleveland,DEF,Cle,1,Jiwei,NaN
86,72,Jalen Hurts,QB,Phi,7,Jiwei,NaN
87,117,Marvin Jones Jr.,WR,Jax,7,Jiwei,NaN
88,45,Justin Jefferson,WR,Min,9,Jiwei,E
89,141,Alexander Mattison,RB,Min,11,Jiwei,NaN


In [29]:
draft_df[draft_df['manager'] == 'Joel']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
96,51,Derek Carr,QB,LV,0,Joel,NaN
97,165,Leonard Fournette,RB,TB,0,Joel,NaN
98,177,Tyler Bass,K,Buf,0,Joel,NaN
99,79,Rashod Bateman,WR,Bal,4,Joel,NaN
100,33,TJ Hockinson,TE,Det,10,Joel,E
101,93,Brandin Cooks,WR,Hou,11,Joel,NaN
102,104,Deebo Samuel,WR,SF,13,Joel,NaN
103,34,Aaron Rodgers,QB,GB,15,Joel,E
104,32,Darrell Henderson Jr.,RB,LAR,16,Joel,E
105,81,Tee Higgins,WR,Cin,28,Joel,NaN


In [30]:
draft_df[draft_df['manager'] == 'Matt']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
112,98,Miami,DEF,Mia,0,Matt,NaN
113,109,Travis Etienne,RB,Jax,0,Matt,NaN
114,120,Jordan Howard,RB,Phi,0,Matt,NaN
115,150,Le'Veon Bell,RB,Bal,0,Matt,NaN
116,156,T.Y. Hilton,WR,Ind,0,Matt,NaN
117,190,Brandon Aiyuk,WR,SF,0,Matt,NaN
118,108,Harrison Butker,K,KC,2,Matt,NaN
119,3,Odell Beckham Jr.,WR,CLE,5,Matt,E
120,4,Jerry Jeudy,WR,DEN,5,Matt,E
121,107,Matthew Stafford,QB,LAR,10,Matt,NaN


In [31]:
draft_df[draft_df['manager'] == 'Rajiv']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
128,70,Eno Benjamin,RB,Ari,0,Rajiv,NaN
129,75,Deonte Harris,WR,NO,0,Rajiv,NaN
130,113,Denver,DEF,Den,0,Rajiv,NaN
131,143,Nyheim Hines,RB,Ind,0,Rajiv,NaN
132,166,Younghoe Koo,K,Atl,0,Rajiv,NaN
133,187,Hunter Henry,TE,NE,0,Rajiv,NaN
134,179,Devin Singletary,RB,Buf,1,Rajiv,NaN
135,171,Marquise Brown,WR,Bal,2,Rajiv,NaN
136,192,Marquez Callaway,WR,NO,3,Rajiv,NaN
137,184,Hunter Renfrow,WR,LV,6,Rajiv,NaN


In [32]:
draft_df[draft_df['manager'] == 'Ron']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
144,123,Joe Burrow,QB,Cin,0,Ron,NaN
145,135,Cordarrelle Patterson,"WR,RB",Atl,0,Ron,NaN
146,158,Los Angeles,DEF,LAC,0,Ron,NaN
147,172,Dawson Knox,TE,Buf,4,Ron,NaN
148,20,Myles Gaskin,RB,MIA,11,Ron,E
149,17,DK Metcalf,WR,SEA,12,Ron,DE
150,111,Dallas Goedert,TE,Phi,13,Ron,NaN
151,110,Corey Davis,WR,NYJ,18,Ron,NaN
152,50,Chase Edmonds,RB,Ari,19,Ron,NaN
153,63,Ja'Marr Chase,WR,Cin,19,Ron,NaN


In [33]:
draft_df[draft_df['manager'] == 'Ryan']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
160,38,New England,DEF,NE,0,Ryan,E
161,114,Daniel Carlson,K,LV,0,Ryan,NaN
162,173,J.K. Dobbins,RB,Bal,0,Ryan,NaN
163,175,Rob Gronkowski,TE,TB,1,Ryan,NaN
164,174,Tony Pollard,RB,Dal,2,Ryan,NaN
165,176,Jamaal Williams,RB,Det,2,Ryan,NaN
166,124,Tampa Bay,DEF,TB,4,Ryan,NaN
167,37,Chase Claypool,WR,PIT,5,Ryan,E
168,118,Laviska Shenault Jr.,WR,Jax,10,Ryan,NaN
169,88,Tom Brady,QB,TB,13,Ryan,NaN


In [34]:
draft_df[draft_df['manager'] == 'Sean']

,pick,player_name,player_position,player_team,salary,manager,keeper_code
176,164,Elijah Moore,WR,NYJ,0,Sean,NaN
177,167,Mark Ingram II,RB,NO,0,Sean,NaN
178,178,Chris Boswell,K,Pit,0,Sean,NaN
179,16,Dak Prescott,QB,DAL,5,Sean,E
180,154,Jaylen Waddle,WR,Mia,5,Sean,NaN
181,188,Dalton Schultz,TE,Dal,9,Sean,NaN
182,90,Mike Williams,WR,LAC,13,Sean,NaN
183,14,AJ Brown,WR,TEN,18,Sean,DE
184,94,Javonte Williams,RB,Den,25,Sean,NaN
185,13,Alvin Kamara,RB,NO,36,Sean,BCDE
